### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
!pip install pytorch_lightning
!pip install optuna
!pip install optuna_integration
!pip install lightning
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 43.2 MB/s eta 0:00:00


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random
import pandas as pd
import optuna

## Carga de datos

In [3]:

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

 ### Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [4]:
tfidfvect = TfidfVectorizer()

In [5]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [6]:
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
y_train = newsgroups_train.target

In [8]:
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [9]:
doc_index = random.choices(range(len(newsgroups_train.data) + 1), k=5)

In [10]:
# midamos la similaridad coseno con todos los documentos de train
def get_cossim(idx, first_similar):
  cossim = cosine_similarity(X_train[idx], X_train)[0]
  mostsim = np.argsort(cossim)[::-1][1:first_similar + 1]
  return mostsim

In [11]:
result = {}
for doc in doc_index:
  fil = get_cossim(doc, 5)
  result[doc] = {}
  result[doc][doc] = newsgroups_train.target_names[y_train[doc]]
  for idx in fil:
    result[doc][idx] = newsgroups_train.target_names[y_train[idx]]

In [12]:
result
# Al validar la categoria inicial y las mejores estimadas, se ve que al menos en las dos primeras mas similares se aproxima en la misma
# categoria

{2905: {2905: 'rec.sport.hockey',
  11298: 'rec.sport.hockey',
  2719: 'rec.sport.hockey',
  11146: 'rec.sport.hockey',
  9760: 'rec.sport.hockey',
  7815: 'rec.sport.baseball'},
 7119: {7119: 'comp.sys.ibm.pc.hardware',
  6552: 'talk.religion.misc',
  5856: 'sci.crypt',
  1055: 'alt.atheism',
  10575: 'sci.crypt',
  9623: 'talk.politics.mideast'},
 5738: {5738: 'alt.atheism',
  5370: 'alt.atheism',
  10836: 'alt.atheism',
  7441: 'talk.politics.guns',
  913: 'alt.atheism',
  6331: 'alt.atheism'},
 4595: {4595: 'sci.space',
  2861: 'sci.space',
  4257: 'sci.space',
  10480: 'sci.med',
  1452: 'talk.politics.guns',
  9718: 'talk.politics.guns'},
 10220: {10220: 'sci.electronics',
  6664: 'sci.electronics',
  4248: 'sci.electronics',
  3611: 'sci.electronics',
  2284: 'misc.forsale',
  8970: 'sci.electronics'}}

In [13]:
pd.set_option('display.max_colwidth', None)
def show_texts(id):
  ids = [ id for id in result[id].keys()]
  texts = [ newsgroups_train.data[id] for id in ids ]
  categories = [ newsgroups_train.target_names[y_train[id]] for id in ids ]
  df = pd.DataFrame({'text': texts, 'category':categories})
  pd.set_option('display.max_colwidth', None)
  return df

In [14]:
df = show_texts(doc_index[0])
df.head(10)
# todos los textos parecen bien clasificados al tocar temas de seguridad

,text,category
0,"\n\tRelying on Canadian tourists and transplanted Northeasterners to\nsupport a team in Miami is crazy; espaecially when you have really deserving \ncities without a team such as San Diego & Milwaukee. I wish the Panthers or\nwhatever their name is well but if they can't sell to Hispanics, they're in\ndeep doo-doo. Already, there are rumors that Tampa may move to Milwaukee.\n",rec.sport.hockey
1,"After reading some of the reports of possible NHL moves to Milwaukee or that\nMilwaukee should have an NHL team, I thought I'd pass along a story I heard\nrecently. This is second hand, so I don't know how true it is, but I have\nno reason to doubt it either.\n\nBradley Center in Milwaukee is home to the Milwaukee Admirals minor leauge\nhockey team. The owner of the Admirals (sorry, I can't remember his name)\neither owns or at least shelled out the majority of the funds to build the\nBradley Center.\n\nSupposedly he was approached by the NHL about an expansion franchise, but \nturned it down because he thought the franchise fee of $50 million was too\nhigh.\n\nLike I said, I don't know whether this story is true or just a rumor, but if\nit's true, don't look for an NHL team in Milwaukee anytime soon. The Admirals\naren't going to be forced out of the building and you won't see an NHL club\nand a minor league club in the same building, especially since the NBA's\nMilwaukee Bucks play there as well.",rec.sport.hockey
2,"The new name is Florida Panthers. \nThe panther is an endangered species, mostly located in the Everglades.\nA couple of years ago, there were license plates made with Panthers on\nthem (part of the revenue were to go to some protection fund).\n\nThe name of the new President of the Panthers should be announced today.\n\nAs of yesterday's paper, Huizenga's new hockey team will take the ice at\nthe Miami Arena this fall. The team has a guaranteed two-year lease with\nthe arena, with four one-year options that could run through 1999.\n\n""It's not our choice"", James Blosser, a lawyer and Huizenga Aid said\nabout ruling out the arena as a long term option. ""The NHL told us we \ncan't stay there. It's not economically feasible.""\n\nOne reason is because the Miami Heat basketball team controls skybox\nand advertising revenue at the arena, reducing the hockey team's profit\npotential.\n\nThe hockey team is attracting arena site proposals from Broward, Dade, \nand Palm Beach counties.\n\nA leading site is vacant land near Joe Robbie Stadium, where residents\n10 days ago agreed to drop their opposition to a hockey arena, ending\nan eight-year battle of wills.\n\nCompliments of the News/Sun-Sentinel.\n\nJim G.",rec.sport.hockey
3,"Well put, Jason. I am not from Wisconsin, but I have close relatives who\nlive in Port Washington (about 30 minutes north of Milwaukee), I visit the\ncity regularly, and I have been in the Bradley four times to see the Admirals\nplay and the NCAA Hockey Championships. It is a beautiful building. The\nPettits and the city like to promote it as the best facility for hockey in\nNorth America.\nAs to what will happen with the Admirals if Milwaukee does acquire a\nfranchise, word is the team will move to Green Bay and play in the Brown\nCounty Arena.\nOnce again, the Admirals are an independent franchise, and the people of\nMilwaukee have been supporting them well. The games I have been to have seen\ncrowds anywhere from 10,000 to 13,000, which are numbers some NHL teams (i.e.\nthe Islanders, Hartford, New Jersey) would be envious of having on some nights.\nPlus the fact that the city is able to support a minor league franchise without\nthe glamour of having an NHL club affiliated to it is testimony to the amount\nof hockey interest exists in the city.\n\nSincerely,\n\nMartin Hill, Rt. 2, Box 155B, Sault Ste. Marie, MI (Home of LSSU: Go Lakers!)",rec.sport.hockey
4,"Well, here it is, NHL in the year 2000.\nI got these from a very reliable source in a dream some years ago and \nalthough 

In [15]:
df = show_texts(doc_index[1])
df.head(10)
# todos los textos parecen bien clasificados al tomar tematicas similares

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [16]:
df = show_texts(doc_index[2])
df.head(10)
# Los primeros dos textos se asocian a la misma tematica, los siguientes si bien no, posiblemente al ser temas mecanicos se halla por eso la relacion

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [17]:
df = show_texts(doc_index[3])
df.head(10)
# Solo dos dos primeros textos clasifican bien en la categoria

,text,category
0,Does anyone know how to size cold gas roll control thruster tanks\nfor sounding rockets?,sci.space
1,">Does anyone know how to size cold gas roll control thruster tanks\n >for sounding rockets?\n\n Well, first you work out how much cold gas you need, then make the\n tanks big enough.\n\nOur sounding rocket payload, with telemetry, guidance, etc. etc. and a\ntelescope cluster, weighs around 1100 pounds. It uses freon jets for\nsteering and a pulse-width-modulated controller for alignment (ie\nduring our eight minutes in space, the jets are pretty much\ncontinuously firing on a ~10% duty cycle or so...). The jets also\nneed to kill residual angular momentum from the spin stabilization, and\nflip the payload around to look at the Sun.\n\nWe have two freon tanks, each holding ~5 liters of freon (I'm speaking\nonly from memory of the last flight). The ground crew at WSMR choose how\nmuch freon to use based on some black-magic algorithm. They have\nextra tank modules that just bolt into the payload stack.\n\nThis should give you an idea of the order of magnitude for cold gas \nquantity. If you really need to know, send me email and I'll try to get you\nin touch with our ground crew people.\n\nCheers,\nCraig\n",sci.space
2,"\nWell, first you work out how much cold gas you need, then make the\ntanks big enough.\n\nWorking out how much cold gas is another problem, depending on\nvehicle configuration, flight duration, thruster Isp (which couples\ninto storage pressure, which may be a factor in selecting tank\nwall thickness etc.)",sci.space
3,\n\ndoes anyone know?\n\n--,sci.med
4,\n\nDo YOU eat all your food cold?\n--,talk.politics.guns
5,"I wrote :\n Nice strawman indeed. The discussion is not about whether there were\ntanks\n used in sixties riots; instead, it is about whether those tanks fired\ntheir\n main guns in one of those riots. You claim they did. That claim is\n ludicrous.\n\nAwesley replied:\n I repeated what I had been told, under what context I had heard it,\n supporting the claim that tanks were indeed used in Detroit in 67.\n\nThe issue has never been whether tanks were used in Detroit in 1967. It\nhas been whether they fired their main guns. You did not merely claim that\ntanks were used--you claimed that they fired their main guns to suppress\nsniper fire and that they were ""quite"" effective at this. You continue to\nback away from this claim and defend something else that nobody is\ndisputing.\n\nAwesley went on:\n I\n spent a few minutes in a library today -- found their computer was\n down and they don't have a card catalog. Anyway, it took about 10\n minutes to find this in _Nightmare in Detroit, A Rebellion and It's\n Victims_ by Sauter and Hines, on page 133, telling of the death of\n Tonia Blanding, age 4.\n \n ""When the tank was fired upon by snipers it turned in the direction\n the shots came from. [...] the fifty-caliber machine gun mounted on the\n tank belched fire into the buildings. After a short round into the front\n of the buildings, the tank guns spit again, tearing apart huge holes out\n of the side of the apartment.""\n \n Well, it's not the main gun. \n\n""Well, it's not the main gun."" Gee, that's only the entire point. Are you\nnow going to admit that you were wrong?\n\n will I see any pictures of tanks firing their main\n guns? Will I see pictures of buildings damaged by the shells? Will I\nread\n the reports of the tank fire? I'll bet you dollar to doughnuts I won't. \n It will take more than second-hand accounts from a few old National Guard\n sergeants shooting the shit to convince me that tanks shelled American\n cities in the Sixties.\n\nAwesley replied:\n Well, if you bothered to read them, it wouldn't take long at all to\n find reports of tank * fire * -- although not necessarily of the main\n guns.\n\nI will never read of tanks firing their main guns in Detroit in the '67\nriots. There is simply no way that such an event could have taken place\nwithout it being common

In [18]:
df = show_texts(doc_index[4])
df.head(10)
# En las primeros 4 resultados corresponden a textos asociatos a Tech, hardware y software, el ultimo no tiene relacion al ser de carros

,text,category
0,"I have an HP 1815 TDR plug-in for an HP180 series scope or mainframe\nthat I'm never going to use (no scope any more). If you're interested\nin it, please let me know. Price? Probably real cheap.\n\nThis notice may have appeared once before. I posted, and it never showed\nup on our local server...\n\n-- \n--------------------------------------------------------------------\n Dave Medin\t\t\tPhone:\t(205) 730-3169 (w)\n SSD--Networking\t\t\t\t(205) 837-1174 (h)\n Intergraph Corp.\n M/S GD3004 \t\tInternet: dtmedin@catbyte.b30.ingr.com\n Huntsville, AL 35894\t\tUUCP: ...uunet!ingr!b30!catbyte!dtmedin\n\n ******* Everywhere You Look (at least around my office) *******",sci.electronics
1,"If anyone out there has an HP180 series scope or mainframe, I\nhave the TDR plug in (the 1810, I believe) for it and have no need\nto keep it. Interested? E-mail me.\n\n-- \n--------------------------------------------------------------------\n Dave Medin\t\t\tPhone:\t(205) 730-3169 (w)\n SSD--Networking\t\t\t\t(205) 837-1174 (h)\n Intergraph Corp.\n M/S GD3004 \t\tInternet: dtmedin@catbyte.b30.ingr.com\n Huntsville, AL 35894\t\tUUCP: ...uunet!ingr!b30!catbyte!dtmedin\n\n ******* Everywhere You Look (at least around my office) *******",sci.electronics
2,"\nTwo things:\n\n\t1. Read your own posts. I was agreeing with Bob. No correction\n\t needed.\n\n\t2. Don't quote stuff out of context.\n\n-- \n--------------------------------------------------------------------\n Dave Medin\t\t\tPhone:\t(205) 730-3169 (w)\n SSD--Networking\t\t\t\t(205) 837-1174 (h)\n Intergraph Corp.\n M/S GD3004 \t\tInternet: dtmedin@catbyte.b30.ingr.com\n Huntsville, AL 35894\t\tUUCP: ...uunet!ingr!b30!catbyte!dtmedin\n\n ******* Everywhere You Look (at least around my office) *******",sci.electronics
3,"\n\nI've never seen a speedometer-reading model. Are you sure? Who makes\nthem? Consider the difficulty of reading the speedo on various makes\nof cars in use... I've seen single beam moving-mode and split beam\nmoving-mode.\n\n-- \n--------------------------------------------------------------------\n Dave Medin\t\t\tPhone:\t(205) 730-3169 (w)\n SSD--Networking\t\t\t\t(205) 837-1174 (h)\n Intergraph Corp.\n M/S GD3004 \t\tInternet: dtmedin@catbyte.b30.ingr.com\n Huntsville, AL 35894\t\tUUCP: ...uunet!ingr!b30!catbyte!dtmedin\n\n ******* Everywhere You Look (at least around my office) *******",sci.electronics
4,"I've got an Eimac 818A/4PR1000A transmitter/linear amplifier tube,\nunused, in original packaging (but opened and inventoried). I'd\nguarantee this tube to operate and be as observation and its\npaperwork say (unused), although I have no transmitter to test it\nwith. Offers?\n\n-- \n--------------------------------------------------------------------\n Dave Medin\t\t\tPhone:\t(205) 730-3169 (w)\n SSD--Networking\t\t\t\t(205) 837-1174 (h)\n Intergraph Corp.\n M/S GD3004 \t\tInternet: dtmedin@catbyte.b30.ingr.com\n Huntsville, AL 35894\t\tUUCP: ...uunet!ingr!b30!catbyte!dtmedin\n\n ******* Everywhere You Look (at least around my office) *******",misc.forsale
5,"A surplus-dealing buddy of mine came up with two emulator pods:\n\n\tHP64220C (for HP 64100 development station). 8086 target\n\tprocessor. DIP head. Does not include board that plugs into\n\tthe 64100.\n\n\tApplied Microsystems 80C186/188 pod, LCC head.\n\nIf you have an interest in either, let me know. They look to\nbe in excellent condition. He doesn't know what to do with them, which\nmay mean that they'll be cheap.\n\n-- \n--------------------------------------------------------------------\n Dave Medin\t\t\tPhone:\t(205) 730-3169 (w)\n SSD--Networking\t\t\t\t(205) 837-1174 (h)\n Intergraph Corp.\n M/S GD3004 \t\tInternet: dtmedin@catbyte.b30.ingr.com\n Huntsville, AL 35894\t\tUUCP: ...uunet!ingr!b30!catbyte!dtmedin\n\n ******* Everywhere You Look (at least around my office) *******",sci.electronics


#Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [19]:
def objective(trial):
  try:
    # Suggest hyperparameters using Optuna
    alpha = trial.suggest_float("alpha", 0.8, 1.2, log=True)
    fit_prior = trial.suggest_categorical("fit_prior", [True, False])
    analyzer = trial.suggest_categorical("analyzer", ["word", "char", "char_wb"])
    ngram_range = trial.suggest_int("ngram_range", 1,4, log=True)
    max_df = trial.suggest_float("max_df", 0.51, 1, log=True)
    min_df = trial.suggest_float("min_df", 0.001, 0.5, log=True)
    norm = trial.suggest_categorical("norm", ["l1", "l2"])
    classifier = trial.suggest_categorical("classifier", ["multinomial", "complement"])
    vectorizer = trial.suggest_categorical("vectorizer", ["count", "tfidf"])

    classifier_dict = {
        "multinomial": MultinomialNB,
        "complement": ComplementNB
    }
    vectorizer_dict = {
        "count": CountVectorizer,
        "tfidf": TfidfVectorizer
    }

    ngram_range_dict = {
        1: (1, 1),
        2: (1, 2),
        3: (2, 2),
        4: (2, 3)
    }
    ngram_range = ngram_range_dict[ngram_range]

    # Create and train a Multinomial Naive Bayes classifier
    clf = classifier_dict[classifier](alpha=alpha, fit_prior=fit_prior)
    clf.fit(X_train, y_train)

    params = {
        "analyzer": analyzer,
        "ngram_range": ngram_range,
        "max_df": max_df,
        "min_df": min_df
    }

    if vectorizer == "tfidf":
      params["norm"] = norm

    tfidfvect = vectorizer_dict[vectorizer](
        **params
    )
    X_train_tfidf = tfidfvect.fit_transform(newsgroups_train.data)
    clf.fit(X_train_tfidf, y_train)
    X_test = tfidfvect.transform(newsgroups_test.data)
    y_test = newsgroups_test.target
    y_pred =  clf.predict(X_test)

    return f1_score(y_test, y_pred, average='macro')

  except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(f"Trial {trial.number} failed due to CUDA out of memory. Trying next combination.")
            trial.report(float('inf'), step=0)
            raise optuna.TrialPruned()
        else:
            raise

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print("Best hyperparameters:", study.best_trial.params)

[I 2024-11-18 23:43:41,423] A new study created in memory with name: no-name-44cfb415-2bc4-4686-a1ca-2e9996b67151
[I 2024-11-18 23:43:55,228] Trial 0 finished with value: 0.25458254993880247 and parameters: {'alpha': 1.1312181159550998, 'fit_prior': True, 'analyzer': 'char', 'ngram_range': 2, 'max_df': 0.8589054323159184, 'min_df': 0.0015332767809541807, 'norm': 'l2', 'classifier': 'multinomial', 'vectorizer': 'count'}. Best is trial 0 with value: 0.25458254993880247.
[I 2024-11-18 23:44:01,835] Trial 1 finished with value: 0.11872319393871193 and parameters: {'alpha': 0.9162869183209829, 'fit_prior': False, 'analyzer': 'word', 'ngram_range': 2, 'max_df': 0.5722468911579305, 'min_df': 0.28932673997805913, 'norm': 'l1', 'classifier': 'multinomial', 'vectorizer': 'tfidf'}. Best is trial 0 with value: 0.25458254993880247.
[I 2024-11-18 23:44:10,089] Trial 2 finished with value: 0.1683328843642073 and parameters: {'alpha': 1.038071347002089, 'fit_prior': False, 'analyzer': 'char', 'ngram_r

Best hyperparameters: {'alpha': 0.8052379784021605, 'fit_prior': False, 'analyzer': 'char_wb', 'ngram_range': 4, 'max_df': 0.5896133311502016, 'min_df': 0.008570081334716203, 'norm': 'l2', 'classifier': 'multinomial', 'vectorizer': 'tfidf'}


### Resultados:

*   alpha: 0.80523797840216052
*   fit_prior: False
*   analyzer: 'char_wb',
*   ngram_range: (2, 3),
*   max_df: 0.5896133311502016,
*   min_df: 0.008570081334716203
*   MultinomialNB (Mejor clasidicador)
*   TfidfVectorizer (Mejor vectorizador)
*   norm: l2

En este proceso no solo se busco los mejores hiperparametros, tambien se busco el mejor clasificador y vectorizador

Nota: la busqueda de un alpha entre  0.8 y 1.2, se debio a experimentos pasados en los que se vio que un alpha cerca a 1 suele dar mejores resultados





 ### Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [20]:
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data).T
feature_names = tfidfvect.get_feature_names_out()
feature_id_dict = {term: id for id, term in enumerate(feature_names)}
id_feature_dict = {id: term for id, term in enumerate(feature_names)}

In [21]:
words = ["car", "apple","voice", "cold", "fire"]

for word in words:
  word_id = feature_id_dict[word]
  related = get_cossim(word_id, 5)
  related = [ id_feature_dict[id] for id in related ]
  print(f"{word} : {related}")


car : ['cars', 'criterium', 'civic', 'owner', 'dealer']
apple : ['_macweek_', 'undisclosed', '5411', 'f83y', 'mb13831fc25']
voice : ['macinators', 'messaging', 'callers', 'intelligable', '56kbps']
cold : ['thruster', 'eat', 'warm', 'leonov', 'fevered']
fire : ['kerosene', 'compound', 'started', 'curiouser', 'arraingement']
